# 103089 - Data mining

<center><img src="media/M-UdL2.png"  width="300" alt="Universitat de Lleida"></center>

## Activity 2: Cosine similarity for movie comparison 

In this exercise you have to implement in a python notebook using the spark framework:

1. The distributed (map/reduce) algorithm of slide "3.7" (in notebook "8-Item-to-Items-globalfiltering-recommenders-py3-sshow.ipynb") for computing the cosine similarity of a set of products with negative and positive ratings, using as input information an RDD (or spark dataframe that is also distributed) with ratings with this format:

- (userID,movieID,rating)

2. The computation of the Cosine Similarity (with the previous algorithm) of all the pairs of movies from the different files you have with this exercise:

- filtered50movies.csv

- filtered100movies.csv

- filtered150movies.csv

- filtered200movies.csv

Each file contains ratings for a different set of movies, but the ones in a smaller file are always a subset of a file with bigger size. We provide files with different size in case you have some memory issues in your computer, so use the biggest file you are able to use, although during "testing" of your code you can of course use the smallest file, or even any smaller subset of the file filtered50movies.csv.

3. Show on the screen the information for the "top 10" most similar pairs, but using the name of the movies you can find in the  movies file.

### Cell 1: Initialization of PySpark

This cell initializes PySpark, a Python library for Spark, a big data processing framework. It sets up the environment, specifies the Python version to use, and creates a SparkContext (sc).

In [ ]:
import pyspark
import os

os.environ["PYSPARK_PYTHON3"] = "python"
spark_home = os.environ.get('SPARK_HOME', None)
print (spark_home)
sc = pyspark.SparkContext.getOrCreate()
sc.setLogLevel('ERROR')
sc

### Cell 2: Loading Data
This cell loads a dataset named Groceries_dataset.csv into a pandas DataFrame (df). It displays the first few rows of the DataFrame to give an overview of the data

In [ ]:
import pandas as pd
df = pd.read_csv("dataset/movies.csv")
df.head()